### 01. Importing libraries

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

### 02. Importing customers dataset

In [2]:
# Creating the path
path = r'C:\Users\veren\Python Data\26-10-2021 Instacart Basket Analysis'

In [3]:
path

'C:\\Users\\veren\\Python Data\\26-10-2021 Instacart Basket Analysis'

In [4]:
# Importing customers data set
df_customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

### 03. Checking the data quality and wrangling the data

In [5]:
# General checks
df_customers.shape

(206209, 10)

In [6]:
df_customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [7]:
# Renaming the columns consistently
df_customers.rename(columns = {'First Name' : 'first_name', 'Surnam' : 'surname','Gender' : 'gender', 'STATE' : 'state', 'Age' : 'age', 'fam_status' : 'family_status'}, inplace = True)

In [8]:
# Check
df_customers.head(2)

,user_id,first_name,surname,gender,state,age,date_joined,n_dependants,family_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285


I chose not to drop any columns.

In [9]:
# Using the describe function to "get to know" the customers data further
df_customers.describe()

,user_id,age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [10]:
# Looking for missing data
df_customers.isnull().sum()

user_id              0
first_name       11259
surname              0
gender               0
state                0
age                  0
date_joined          0
n_dependants         0
family_status        0
income               0
dtype: int64

I decided to not do anything about the missing first_name values because as long as I have the surname values the data is important and unique.

In [11]:
# Checking for columns with mixed data types in the customers dataset to prevent errors

for col in df_customers.columns.tolist():
  weird = (df_customers[[col]].applymap(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customers[weird]) > 0:
    print (col)


first_name


In [12]:
df_customers['first_name'].dtype

dtype('O')

In [13]:
# Converting the entries of the first_name column into a string
df_customers['first_name']= df_customers['first_name'].astype('str')

### 04. Combining the previous Instacart data with the new customers data

In [14]:
# Importing the most recent instacart data
df_ords_prods_merge = pd.read_pickle(r'C:\Users\veren\Python Data\26-10-2021 Instacart Basket Analysis\02 Data\Prepared Data\ords_prods_merge_GDAV_whole.pkl')

In [15]:
df_ords_prods_merge.shape

(30356128, 25)

In [16]:
df_ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,eval_set,order_number,order_dow,...,price_range_loc,busiest day,busiest days,busiest_period_of_day,max_order,loyalty_flag,spending,spending_flag,frequency,frequency_flag
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,prior,28,6,...,Mid-range product,Regularly busy,Regularly busy,Most Orders,32,Regular customer,6.769286,Low Spender,8.0,Frequent customer
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,prior,30,6,...,Mid-range product,Regularly busy,Regularly busy,Most Orders,32,Regular customer,6.769286,Low Spender,8.0,Frequent customer
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,prior,2,0,...,Mid-range product,Busiest day,Busiest days,Average Orders,5,New customer,8.071613,Low Spender,8.0,Frequent customer
3,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,prior,3,4,...,Mid-range product,Least busy,Slowest days,Most Orders,3,New customer,4.864103,Low Spender,9.0,Frequent customer
4,1,Chocolate Sandwich Cookies,61,19,5.8,1701441,777,prior,16,1,...,Mid-range product,Regularly busy,Busiest days,Average Orders,26,Regular customer,6.906422,Low Spender,11.0,Regular customer


In [17]:
# Removing unnecessary data ("eval_set")

df_ords_prods_merge.drop(columns = ['eval_set'], inplace = True)

In [18]:
# Check
df_ords_prods_merge.columns

Index(['product_id', 'product_name', 'aisle_id', 'department_id', 'prices',
       'order_id', 'user_id', 'order_number', 'order_dow', 'order_hour_of_day',
       'days_since_prior_order', 'add_to_cart_order', 'reordered', '_merge',
       'price_range_loc', 'busiest day', 'busiest days',
       'busiest_period_of_day', 'max_order', 'loyalty_flag', 'spending',
       'spending_flag', 'frequency', 'frequency_flag'],
      dtype='object')

In [19]:
# Removing _merge column
df_ords_prods_merge.drop(columns = ['_merge'], inplace = True)

In [20]:
#Check 
df_ords_prods_merge.columns

Index(['product_id', 'product_name', 'aisle_id', 'department_id', 'prices',
       'order_id', 'user_id', 'order_number', 'order_dow', 'order_hour_of_day',
       'days_since_prior_order', 'add_to_cart_order', 'reordered',
       'price_range_loc', 'busiest day', 'busiest days',
       'busiest_period_of_day', 'max_order', 'loyalty_flag', 'spending',
       'spending_flag', 'frequency', 'frequency_flag'],
      dtype='object')

In [21]:
# Renaming "busiest day" and "busiest days" for consistency
df_ords_prods_merge.rename(columns = {'busiest days' : 'busiest_days', 'busiest day': 'busiest_day'}, inplace = True)

In [22]:
# Check
df_ords_prods_merge.columns

Index(['product_id', 'product_name', 'aisle_id', 'department_id', 'prices',
       'order_id', 'user_id', 'order_number', 'order_dow', 'order_hour_of_day',
       'days_since_prior_order', 'add_to_cart_order', 'reordered',
       'price_range_loc', 'busiest_day', 'busiest_days',
       'busiest_period_of_day', 'max_order', 'loyalty_flag', 'spending',
       'spending_flag', 'frequency', 'frequency_flag'],
      dtype='object')

In [23]:
# Checking the datatypes of df_ords_prods_merge to make sure the user_id column has the same data type in both data sets
df_ords_prods_merge.dtypes

product_id                  int64
product_name               object
aisle_id                    int64
department_id               int64
prices                    float64
order_id                    int64
user_id                     int64
order_number                int64
order_dow                   int64
order_hour_of_day           int64
days_since_prior_order    float64
add_to_cart_order           int64
reordered                   int64
price_range_loc            object
busiest_day                object
busiest_days               object
busiest_period_of_day      object
max_order                   int64
loyalty_flag               object
spending                  float64
spending_flag              object
frequency                 float64
frequency_flag             object
dtype: object

In [24]:
df_customers.dtypes

user_id           int64
first_name       object
surname          object
gender           object
state            object
age               int64
date_joined      object
n_dependants      int64
family_status    object
income            int64
dtype: object

The columns "user_id" is int64 in both data sets so merging both data frames on this column should work.

### 05. Merging the two data sets

In [25]:
# Combining the customer data with the previous Instacart data
df_ords_prods_cust_merged = df_ords_prods_merge.merge(df_customers, on = 'user_id')

In [26]:
df_ords_prods_cust_merged.shape

(30356128, 32)

In [27]:
df_ords_prods_cust_merged.head(100)

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,order_dow,order_hour_of_day,...,frequency_flag,first_name,surname,gender,state,age,date_joined,n_dependants,family_status,income
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,Frequent customer,Charles,Cox,Male,Minnesota,81,8/1/2019,1,married,49620
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,Frequent customer,Charles,Cox,Male,Minnesota,81,8/1/2019,1,married,49620
2,907,Premium Sliced Bacon,106,12,20.0,2254091,138,10,5,14,...,Frequent customer,Charles,Cox,Male,Minnesota,81,8/1/2019,1,married,49620
3,1000,Apricots,18,10,12.9,505689,138,9,6,12,...,Frequent customer,Charles,Cox,Male,Minnesota,81,8/1/2019,1,married,49620
4,3265,Mini Seedless Watermelon Pack,32,4,1.7,960220,138,17,5,14,...,Frequent customer,Charles,Cox,Male,Minnesota,81,8/1/2019,1,married,49620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,31805,Gourmet Tomato Medley,32,4,5.2,2526903,138,14,0,15,...,Frequent customer,Charles,Cox,Male,Minnesota,81,8/1/2019,1,married,49620
96,31860,Foil Sheets,85,17,11.7,1593782,138,15,6,11,...,Frequent customer,Charles,Cox,Male,Minnesota,81,8/1/2019,1,married,49620
97,32478,Reduced Fat 2% Milk,84,16,13.9,2206168,138,27,3,18,...,Frequent customer,Charles,Cox,Male,Minnesota,81,8/1/2019,1,married,49620
98,32689,Romaine Hearts,123,4,2.9,960220,138,17,5,14,...,Frequent customer,Charles,Cox,Male,Minnesota,81,8/1/2019,1,married,49620


In [28]:
# Exporting the data as pkl
path

'C:\\Users\\veren\\Python Data\\26-10-2021 Instacart Basket Analysis'

In [29]:
df_ords_prods_cust_merged.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_cust_merged_1.pkl'))